In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

import numpy as np

In [6]:
# Load the datasets
gw1 = pd.read_csv('../Data/2023-24/gws/gw1.csv')
gw2 = pd.read_csv('../Data/2023-24/gws/gw2.csv')
gw3 = pd.read_csv('../Data/2023-24/gws/gw3.csv')
gw4 = pd.read_csv('../Data/2023-24/gws/gw4.csv')
gw5 = pd.read_csv('../Data/2023-24/gws/gw5.csv')
gw6 = pd.read_csv('../Data/2023-24/gws/gw6.csv')
gw7 = pd.read_csv('../Data/2023-24/gws/gw7.csv')
gw8 = pd.read_csv('../Data/2023-24/gws/gw8.csv')
gw9 = pd.read_csv('../Data/2023-24/gws/gw9.csv')
gw10 = pd.read_csv('../Data/2023-24/gws/gw10.csv')
gw11 = pd.read_csv('../Data/2023-24/gws/gw11.csv')
gw12 = pd.read_csv('../Data/2023-24/gws/gw12.csv')
gw13 = pd.read_csv('../Data/2023-24/gws/gw13.csv')
gw14 = pd.read_csv('../Data/2023-24/gws/gw14.csv')
gw15 = pd.read_csv('../Data/2023-24/gws/gw15.csv')
gw16 = pd.read_csv('../Data/2023-24/gws/gw16.csv')
gw17 = pd.read_csv('../Data/2023-24/gws/gw17.csv')


In [7]:
gw_data_frames = {
    1: gw1,
    2: gw2,
    3: gw3,
    4: gw4,
    5: gw5,
    6: gw6,
    7: gw7,
    8: gw8,
    9: gw9,
    10: gw10,
    11: gw11,
    12: gw12,
    13: gw13,
    14: gw14,
    15: gw15,
    16: gw16,
    17: gw17
}

In [8]:
# Add a 'gw' column to each DataFrame to indicate the game week, then concatenate them into one DataFrame
for gw, df in gw_data_frames.items():
    df['gw'] = gw

# Combine all the DataFrames
combined_df = pd.concat(gw_data_frames.values(), ignore_index=True)

# Display the shape and the first few rows of the combined DataFrame to confirm success
combined_df.shape, combined_df.head()

((12243, 41),
                      name position           team   xP  assists  bonus  bps  \
 0             Femi Seriki      DEF  Sheffield Utd  0.5        0      0    0   
 1        Jack Hinshelwood      MID       Brighton  1.5        0      0    0   
 2            Jadon Sancho      MID        Man Utd  3.0        0      0    4   
 3  Rhys Norrington-Davies      DEF  Sheffield Utd  0.1        0      0    0   
 4           Vitaly Janelt      MID      Brentford  2.1        0      0    6   
 
    clean_sheets  creativity  element  ...  team_h_score  threat  total_points  \
 0             0         0.0      653  ...             0     0.0             0   
 1             0         0.0      621  ...             4     0.0             0   
 2             0        11.3      397  ...             1     8.0             1   
 3             0         0.0      487  ...             0     0.0             0   
 4             0        11.5      105  ...             2    17.0             2   
 
    transf

In [10]:
# Example for data cleaning
combined_df.dropna(inplace=True)  # Simple missing value handling

# Example for feature engineering: Calculate a simple form metric
combined_df['form'] = combined_df.groupby('name')['total_points'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())


In [19]:
# Isolating points for week 17
week17_points = combined_df[combined_df['gw'] == 17][['name', 'total_points']].copy()

# Renaming for clarity
week17_points.rename(columns={'total_points': 'future_points'}, inplace=True)

In [21]:
combined_df = combined_df.merge(week17_points, on='name', how='left')

In [24]:
# Assuming `combined_df` is your DataFrame and it's already prepared with necessary features and target variable

# Feature selection (replace with actual feature names)
features = ['form', 'position', 'team', 'goals_conceded', 'goals_scored','assists','minutes','saves', 'clean_sheets', 'yellow_cards', 'red_cards']

# Encode categorical features
categorical_features = ['position', 'team']
numerical_features = ['form', 'goals_conceded', 'goals_scored', 'assists', 'minutes', 'saves', 'clean_sheets', 'yellow_cards', 'red_cards']

# One-hot encode categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])